MAIN CODE

In [6]:
import requests
import datetime

thunder1=[] #name
thunder2=[] #price
location = []
description = []
time = []


for page_number in range(0,1): 

    url = "https://api.bunjang.co.kr/api/1/find_v2.json?q=%EC%95%84%EC%9D%B4%ED%8F%B011&order=score&page={0}&request_id=2022519113331&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4".format(page_number)
    res = requests.get(url)
    res.encoding=None
    data = res.json()
    
    for d in data['list']:
        name = d['name']
        loc = d['location']
        price = int(d['price'])
        pid = d['pid']
        unix = str(datetime.datetime.fromtimestamp(d['update_time']))
        unix = unix[0:7]

        
        excepts = ["케이스","삽니다", "매입", "구매", "구입","프로","Pro","pro", "미니", "PRO", "11PRO"]
        targets = ['배터리', '판매', '팔아요']

        if (price >= 50000) & (price < 1600000) ==True :
            if ( d['ad']==False ):
                if any(x in d["name"] for x in targets) and not any(x in d['name'] for x in excepts):
                    thunder1.append(d['name'])
                    thunder2.append(price)
                    time.append(unix)
                    loc = loc.split(' ')
                    loc = loc[0]
                    # location.append(loc)
                    if len(loc) == 0:
                        location.append('NULL')
                    else:
                        location.append(loc)

                    url2 = 'https://api.bunjang.co.kr/api/1/product/{0}/detail_info.json?version=4'.format(pid)
                    res = requests.get(url2)
                    res.encoding=None
                    data = res.json()
                
                    description.append(data['item_info']['description'])
                    


                    # print(name, price, unix, loc)

        else:
            pass

# print(thunder1,thunder2, time, location )


Desc 에서 배터리 효율 뽑아내기

In [7]:
import re
from bs4 import BeautifulSoup

li = []



for i in range(len(description)):
    text = description[i]
    regex = '배터리\s*.*\d{2,3}'
    pattern =  re.compile(regex)
    res = pattern.findall(text)
    
    regex2 = '\d{2,3}'
    pattern2 = re.compile(regex2)
    text = str(res)
    res2 = pattern2.findall(text)
    if len(res2) == 0:
        li.append('NULL')
    else:
        li.append(res2[0])

print(li)



'''
for i in range(len(li)):    
    text = li[i]
    regex2 = '\d{2,3}'
    pattern2 = re.compile(regex2)
    res2 = pattern2.findall(text)
    li1.append(res2)
'''
#print(li1)

['100']


"\nfor i in range(len(li)):    \n    text = li[i]\n    regex2 = '\\d{2,3}'\n    pattern2 = re.compile(regex2)\n    res2 = pattern2.findall(text)\n    li1.append(res2)\n"

용량 빼내기

In [8]:
size = []

for i in range(len(description)):
    text = description[i]
    regex = '64|128'
    pattern =  re.compile(regex)
    res = pattern.findall(text)
    
    if len(res) == 0:
        size.append('NULL')
    else:
        size.append(res[0])
print(size)

['64']


판다스로 데이터프레임 만들기

In [9]:
import pandas as pd
df = pd.DataFrame({'name' : thunder1,
                 'price' : thunder2,
                 'time' : time,
                  'loction' : location,
                  'battery' : li,
                  'size': size
                  })
df

,name,price,time,loction,battery,size
0,아이폰11 64 배터리100% 옐로우 ) 63666,390000,2022-05,인천광역시,100,64


데이터 엑셀파일로 추출

In [43]:
from pandas import ExcelWriter
df.to_excel('C:/Users/admin/Desktop/big/thunder.xlsx', index=False)